<a href="https://colab.research.google.com/github/Tejjus/movie-graph-nlp-chatbot/blob/main/graph_nlp_query_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import google.generativeai as palm
import base64
import json
import gradio as gr
from neo4j import GraphDatabase

In [3]:
from google.colab import userdata
palm.configure(api_key = userdata.get("palm_api_key"))

In [4]:
def get_answer(input):

  defaults = {
    'model': 'models/text-bison-001',
    'temperature': 0.7,
    'candidate_count': 1,
    'top_k': 40,
    'top_p': 0.95,
    'max_output_tokens': 1024,
    'stop_sequences': [],
    'safety_settings': [{"category":"HARM_CATEGORY_DEROGATORY","threshold":1},{"category":"HARM_CATEGORY_TOXICITY","threshold":1},{"category":"HARM_CATEGORY_VIOLENCE","threshold":2},{"category":"HARM_CATEGORY_SEXUAL","threshold":2},{"category":"HARM_CATEGORY_MEDICAL","threshold":2},{"category":"HARM_CATEGORY_DANGEROUS","threshold":2}],
  }

  prompt = f"""You are an expert in converting English questions to Neo4j Cypher Graph code! The Graph has following Node Labels - Movie, Person! the Movie Node has the following properties released, tagline, title. The Person node has properties such as name & born. The Neo4j Graph has the following Relationship types ACTED_IN, DIRECTED, FOLLOWS, PRODUCED, REVIEWED, WROTE!

  All relationships ACTED_IN, DIRECTED, FOLLOWS, PRODUCED, REVIEWED, WROTE start from Person to Movie and not the other way around.

  For example,
  Example 1 - List down 5 movies that released after the year 2000, the Cypher command will be something like this
  ``` MATCH (m:Movie)
  WHERE m.released > 2000
  RETURN m LIMIT 5
  ```

  Example 2 - Get all the people who acted in a movie that was released after 2010.
  ```
  MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
  WHERE m.released > 2010
  RETURN p,r,m
  ```

  Example 3 - Name the Director of the movie Apollo 13?
  ```
  MATCH (m:Movie)<-[:DIRECTED]-(p:Person)
  WHERE m.title = 'Apollo';
  RETURN p.name
  ```

  Do not include ``` and \n in the output

  {input}"""
  response = palm.generate_text(**defaults,prompt=prompt)

  return response.result

In [5]:
get_answer("Which movie did Emil Eifrem act in?")

"MATCH (p:Person {name:'Emil Eifrem'})-[a:ACTED_IN]->(m:Movie) RETURN m.title"

In [6]:
driver = GraphDatabase.driver("neo4j+s://a18c3962.databases.neo4j.io",
                              auth=("neo4j",
                                    "y2-2WQTJ_nNdqnzJou0R2rAopWCQmDA6IhB0yo1gDYo"))

In [7]:
import re

In [8]:
def extract_query_and_return_key(input_query_result):
    slash_n_pattern = r'[ \n]+'
    ret_pattern = r'RETURN\s+(.*)'
    replacement = ' '

    cleaned_query = re.sub(slash_n_pattern, replacement, input_query_result)
    if cleaned_query:
        match = re.search(ret_pattern, cleaned_query)
        if match:
            extracted_string = match.group(1)
        else:
            extracted_string = ""
    return cleaned_query, extracted_string

In [9]:
extract_query_and_return_key(get_answer("Who were the actors in the movie V for Vendetta"))

("MATCH (p:Person)-[r:ACTED_IN]->(m:Movie) WHERE m.title = 'V for Vendetta' RETURN p.name",
 'p.name')

In [10]:
def format_names_with_ampersand(names):
    if len(names) == 0:
        return ""
    elif len(names) == 1:
        return names[0]
    else:
        formatted_names = ", ".join(names[:-1]) + " & " + names[-1]
        return formatted_names

In [11]:
format_names_with_ampersand(["Sachin","Virat","Rahul"])

'Sachin, Virat & Rahul'

In [12]:
def run_cypher_on_neo4j(inp_query, inp_key):
    out_list = []
    with driver.session() as session:
        result = session.run(inp_query)
        for record in result:
            out_list.append(record[inp_key])
    driver.close()
    if len(out_list) > 1:
        return format_names_with_ampersand(out_list)
    else:
        return out_list[0]

In [13]:
query_1, string_1 = extract_query_and_return_key(get_answer("Who were the actors in the movie V for Vendetta"))

run_cypher_on_neo4j (query_1, string_1)

'John Hurt, Stephen Rea, Natalie Portman, Hugo Weaving & Ben Miles'

In [14]:
def generate_and_exec_cypher(input_query):
    gen_query, gen_key = extract_query_and_return_key(get_answer(input_query))
    return run_cypher_on_neo4j(gen_query, gen_key)

In [15]:
def chatbot(input, history=[]):
    output = str(generate_and_exec_cypher(input))
    history.append((input, output))
    return history, history

In [16]:
gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://26b289c330270f2770.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<ipython-input-12-6d234cc3d37d>:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 456, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1522, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1144, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packag

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://26b289c330270f2770.gradio.live
